In [1]:
import pandas as pd
import numpy as np
import cv2
import keras

2023-01-16 13:48:52.204686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 13:48:53.259495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/karan/Secondary/dev/ml/.venv/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-16 13:48:53.259521: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-16 13:48:53.328443: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when o

In [2]:
CSV_FILE = r"data/csv/road_classification.csv"

features = pd.read_csv(CSV_FILE)

features.head(10)

,file_name,normal,negative,positive
0,data/final_data/1.jpg,0,2,0
1,data/final_data/10.jpg,0,1,0
2,data/final_data/100.jpg,1,0,0
3,data/final_data/101.jpg,1,0,0
4,data/final_data/102.jpg,1,0,0
5,data/final_data/103.jpg,0,3,0
6,data/final_data/104.jpg,1,0,0
7,data/final_data/105.jpg,1,0,0
8,data/final_data/106.jpg,1,0,0
9,data/final_data/107.jpg,1,0,0


In [5]:
features.isna().sum()

file_name    0
normal       0
negative     0
positive     0
dtype: int64

In [8]:
from sklearn.utils import shuffle

img_names = features['file_name']
labels = features.drop('file_name', axis=1)

img_names_shuffled, labels_shuffled = shuffle(img_names, labels)

In [9]:
from sklearn.model_selection import train_test_split

X_train_img, X_test_img, y_train, y_test = train_test_split(
    img_names_shuffled, labels_shuffled,
    test_size=0.2,
    random_state=101
)

In [10]:
train_class = y_train[['normal', 'negative', 'positive']]
test_class = y_test[['normal', 'negative', 'positive']]

In [13]:
y_train.shape

(260, 3)

In [14]:
def get_img_by_loc(location: str) -> cv2.Mat:
    img = cv2.imread(location)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255
    img = cv2.resize(img, (800,800))
    return img

In [16]:
class ImageGenerator(keras.utils.Sequence):

    def __init__(self, img_files, img_class_num, batch_size = 32) -> None:
        self.img_files = img_files
        self.img_class_num = img_class_num
        self.batch_size = batch_size

    
    def __len__(self):
        return (np.ceil(len(self.img_files) / float(self.batch_size))).astype(np.int)
    
    
    def __getitem__(self, idx):
        batch_x = self.img_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = [np.array(self.img_class_num[idx * self.batch_size : (idx + 1) * self.batch_size])]

        return np.array([
            get_img_by_loc(f'{file_name}')
            for file_name in batch_x
        ]), batch_y

In [17]:
batch_size = 8

train_batch_gen = ImageGenerator(X_train_img, train_class, batch_size)
test_batch_gen = ImageGenerator(X_test_img, test_class, batch_size)

In [18]:
import tensorflow
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input, Activation, Add, BatchNormalization

In [19]:
input_ = Input(shape=(800,800,1), name="Input Layer")

conv_1 = Conv2D(32, kernel_size= (4, 4), name="conv_1")(input_)
act_1  = Activation("relu", name="act_1")(conv_1)
pool_1 = MaxPooling2D(pool_size=(8, 8), strides=(1,1), padding="valid", name="pool_1")(act_1)

conv_2 = Conv2D(64, kernel_size = (8, 8), name = "conv_2")(pool_1)
act_2 = Activation("relu", name = "act_2")(conv_2)
pool_2 = MaxPooling2D(pool_size = (4, 4), strides = (1, 1), padding = "valid", name = "pool_2")(act_2)

conv_3 = Conv2D(32, kernel_size = (4, 4), name = "conv_3")(pool_2)
act_3 = Activation("relu", name = "act_3")(conv_3)
pool_3 = MaxPooling2D(pool_size = (8, 8), strides = (1, 1), padding = "valid", name = "pool_3")(act_3)

conv_4 = Conv2D(16, kernel_size = (4, 4), name = "conv_4")(pool_3)
act_4 = Activation("relu", name = "act_4")(conv_4)
pool_4 = MaxPooling2D(pool_size = (4, 4), strides = (1, 1), padding = "valid", name = "pool_4")(act_4)

conv_5 = Conv2D(16, kernel_size = (8, 8), name = "conv_5")(pool_4)
act_5 = Activation("relu", name = "act_5")(conv_5)
pool_5 = MaxPooling2D(pool_size = (4, 4), strides = (1, 1), padding = "valid", name = "pool_5")(act_5)

flat_1 = Flatten(name = "flat_1")(pool_5)

dense_1 = Dense(128, activation = "relu", name = "dense_1")(flat_1)
batch_1 = BatchNormalization(name="batch_1")(dense_1)
dense_2 = Dense(64, activation="relu", name="dense_2")(batch_1)
batch_2 = BatchNormalization(name="batch_2")(dense_2)
dense_3 = Dense(32, activation="relu", name="dense_3")(batch_2)
drop_1 = Dropout(rate=0.4, name="drop_1")(dense_3)
dense_4 = Dense(16, activation="relu", name="dense_4")(drop_1)
drop_2 = Dropout(rate=0.4, name="drop_2")(dense_4)
dense_5 = Dense(8, activation="relu", name="dense_5")(drop_2)
img_class = Dense(3, activation="softmax", name="img_class")(dense_5)

model = Model(inputs = input_, outputs = img_class)
opt = tensorflow.keras.optimizers.SGD(learning_rate=0.01)

model.compile(
    loss = {
        "img_class" : "categorical_crossentropy"
    },
    optimizer=opt,
    metrics=['accuracy']
)

model.summary()


2023-01-16 14:34:55.097368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 14:34:55.098100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/karan/Secondary/dev/ml/.venv/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-16 14:34:55.098332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/karan/Secondary/dev/ml/.venv/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-16 14:34:55.098512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li